In [ ]:
%store -r
import time
import os,json
import requests
from dkube.sdk import *
from dkube.sdk.api import DkubeApi
from dkube.sdk.rsrcs import DkubeModelmonitor
from dkube.sdk.rsrcs.modelmonitor import DatasetClass,ModelType,DriftAlgo, DataType
from dkube.sdk.rsrcs.modelmonitor import DatasetFormat,DkubeModelmonitoralert, TimeZone

In [ ]:
api = DkubeApi(URL=os.getenv('DKUBE_URL'),token=os.getenv("DKUBE_USER_ACCESS_TOKEN"))

### Importing deployment

In [ ]:
def wait_for_deployment_running(deployment_id):
    status = None
    inference_url, inference = None, None
    while True:
        data = api.get_deployment(deployment_id)
        status = data.data.inferenceservice_deployment.parameters.generated.status.state
        inference = data.data.inferenceservice_deployment.parameters.inference
        inference_url = data.data.inferenceservice_deployment.parameters.generated.details.serving.servingurl
        if status == "RUNNING":
            break
        print("waiting for deployment to reach running state")
        time.sleep(api.wait_interval)
    return inference, inference_url

In [ ]:
text_file = open("transform-data.py", "r")
#read whole file to a string
script = text_file.read()
#close file
text_file.close()

with open('thresholds.json') as f:
    thresholds = json.load(f)

In [ ]:
while True:
    DEPLOYMENT_ID = api.get_deployment_id(name=MONITOR_NAME)
    if DEPLOYMENT_ID:
        break
    print("waiting for deployment to come up")
    time.sleep(api.wait_interval)
inference, INFERENCE_URL = wait_for_deployment_running(DEPLOYMENT_ID)
if not inference.enable_logs:
    deployment_url = f"{DKUBE_URL}/dkube/v2/controller/users/{DKUBEUSERNAME}/inference/{MONITOR_NAME}/update"
    payload = {'version': inference.version, "enable_logs": True}
    payload = json.dumps(payload)
    resp = requests.put(deployment_url, headers={"Authorization": f"Bearer {TOKEN}", "content-type":"application/keyauth.api.v1+json"}, data= payload,verify=False)
    print(resp.status_code)
    if resp.status_code == 200:
        _, INFERENCE_URL = wait_for_deployment_running(DEPLOYMENT_ID)
%store INFERENCE_URL
%store DEPLOYMENT_ID

In [ ]:
training_data = f'{DKUBE_BASE_DATASET}:'+ DKUBEUSERNAME
train_data_version = 'v1:'+api.get_dataset_versions(
    DKUBEUSERNAME,
    DKUBE_BASE_DATASET)[0]['version']['uuid']
prediction_data = LIVE_DATASET + ":"+ DKUBEUSERNAME
labelled_data = LIVE_DATASET + ":"+ DKUBEUSERNAME
predict_data_format = "cloudeventlogs"

### Model Monitor

In [ ]:
mm=DkubeModelmonitor(deployemnt_id = DEPLOYMENT_ID)

In [ ]:
mm.update_modelmonitor_basics(model_type=ModelType.Regression.value, 
                               input_data_type=DataType.Tabular.value,
                               data_timezone=TimeZone.UTC.value)

In [ ]:
mm.add_thresholds(thresholds=thresholds)

## Health Monitoring

In [ ]:
deployment = api.get_deployment(DEPLOYMENT_ID)
if ((deployment.data.imported_deployment and deployment.data.imported_deployment.cluster)
    or
    not deployment.data.imported_deployment):
    mm.update_deployment_monitoring_details(enabled=True, frequency=1, collect_metrics=True)

## Add Drift monitoring details

In [ ]:
mm.update_drift_monitoring_details(enabled=True,frequency=5,algorithm='auto')

### Add Train, Prediction, and Labelled Datasets

In [ ]:
mm.add_datasources(data_class=str(DatasetClass.Train),name=training_data,
                   data_format="tabular",
                   version=train_data_version,transformer_script = script)

mm.add_datasources(data_class=str(DatasetClass.Predict),name=prediction_data,
                   s3_subpath=DEPLOYMENT_ID, data_format=predict_data_format,
                   date_suffix="yyyy/mm/dd/hh")
mm.add_datasources(data_class=str(DatasetClass.Labelled),name=labelled_data,
                   data_format="tabular", s3_subpath=DEPLOYMENT_ID + "/livedata",
                   predict_col="charges",groundtruth_col="GT_target",timestamp_col="timestamp")

### Performance Monitoring

In [ ]:
mm.update_performance_monitoring_details(enabled=True,source_type="labelled_data",frequency=5)

### Create Model monitor

In [ ]:
api.modelmonitor_create(mm,wait_for_completion=True)

### Extracting id of the Model Monitor

In [ ]:
# The function below can be used to fetch model monitor ID by name.
# The monitor id will be same as deployment id.
# id = api.modelmonitor_get_id(MONITOR_NAME)
id = DEPLOYMENT_ID
print(id)

### Schema update

In [ ]:
api.modelmonitor_update_schema(id,label='charges',schema_class='continuous',schema_type="prediction_output",selected=False)
api.modelmonitor_update_schema(id,label='unique_id',schema_class='continuous',schema_type="row_id",selected=False)
api.modelmonitor_update_schema(id,label='timestamp',schema_class='continuous',schema_type="timestamp",selected=False)

## age and bmi to continuous
api.modelmonitor_update_schema(id,label='age',schema_class='continuous',schema_type='input_feature',selected=True)
api.modelmonitor_update_schema(id,label='bmi',schema_class='continuous',schema_type='input_feature',selected=True)

## select these features
api.modelmonitor_update_schema(id,label='sex',schema_type='input_feature',schema_class='categorical',selected=True)
api.modelmonitor_update_schema(id,label='children',schema_type='input_feature',schema_class='categorical',selected=True)
api.modelmonitor_update_schema(id,label='smoker',schema_type='input_feature',schema_class='categorical',selected=True)
api.modelmonitor_update_schema(id,label='region',schema_type='input_feature',schema_class='categorical',selected=True)

### Add alerts

In [ ]:
alert = DkubeModelmonitoralert(name='age_alert')
alert.update_alert(alert_class = 'feature_drift',feature='age',threshold=0.02)
api.modelmonitor_add_alert(id,alert)

### Start the model monitor

In [ ]:
api.modelmonitor_start(id)

### Retraining / Rebaselining Model Monitor

### Cleanup

In [ ]:
CLEANUP = False
if CLEANUP:
    from time import sleep
    RETRIES = 4
    while RETRIES:
        mm = api.modelmonitor_get(id)
        if mm["status"] and mm["status"]["state"].lower() != "active":
            break
        elif mm["status"] and mm["status"]["state"].lower() == "active":
            api.modelmonitor_stop(id)
        RETRIES -= 1
        sleep(5)
    else:
        raise TimeoutError("modelmonitor failed to stopped")
    api.modelmonitor_delete(id)
    